## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-07-01-12-08 +0000,bbc,Inner London to lose out in funding rebalance ...,https://www.bbc.com/news/articles/cy4dv2dgjeko...
1,2025-08-07-01-08-27 +0000,nyt,United Airlines System Outage Leads to Grounde...,https://www.nytimes.com/2025/08/06/business/un...
2,2025-08-07-01-05-56 +0000,nyt,A Sergeant Is Accused of Shooting 5 Soldiers i...,https://www.nytimes.com/2025/08/06/us/fort-ste...
3,2025-08-07-01-05-55 +0000,nyt,Federal Agents in L.A. Conduct a Big Immigrati...,https://www.nytimes.com/2025/08/06/us/los-ange...
4,2025-08-07-01-05-29 +0000,bbc,Badenoch reveals how she told on exam cheat as...,https://www.bbc.com/news/articles/c80d7l03137o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2321/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
47,trump,55
165,india,16
59,new,13
173,will,12
126,texas,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
189,2025-08-06-15-28-28 +0000,bbc,Trump-Modi ties hit rock bottom with new tarif...,https://www.bbc.com/news/articles/c15lqe7v302o...,118
118,2025-08-06-19-46-31 +0000,nypost,"Trump plans sitdown with Putin, Zelensky in fi...",https://nypost.com/2025/08/06/us-news/trump-pl...,105
24,2025-08-07-00-00-00 +0000,wsj,Democrats are adding potentially risky Epstein...,https://www.wsj.com/politics/policy/epstein-fi...,104
207,2025-08-06-14-14-46 +0000,nypost,Trump slaps additional 25% tariff on India ove...,https://nypost.com/2025/08/06/us-news/trump-sl...,103
76,2025-08-06-21-56-54 +0000,wapo,"Trump raises tariffs on India to 50 percent, c...",https://www.washingtonpost.com/business/2025/0...,101


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
189,118,2025-08-06-15-28-28 +0000,bbc,Trump-Modi ties hit rock bottom with new tarif...,https://www.bbc.com/news/articles/c15lqe7v302o...
261,52,2025-08-06-10-00-00 +0000,latimes,Chabria: Newsom vows Texas will be 'neutered' ...,https://www.latimes.com/california/story/2025-...
118,50,2025-08-06-19-46-31 +0000,nypost,"Trump plans sitdown with Putin, Zelensky in fi...",https://nypost.com/2025/08/06/us-news/trump-pl...
24,47,2025-08-07-00-00-00 +0000,wsj,Democrats are adding potentially risky Epstein...,https://www.wsj.com/politics/policy/epstein-fi...
17,40,2025-08-07-00-10-12 +0000,nypost,Out-of-control driver accused of killing 4 Pep...,https://nypost.com/2025/08/06/us-news/fraser-m...
161,38,2025-08-06-16-56-15 +0000,nypost,Italy approves $15.5 billion project to build ...,https://nypost.com/2025/08/06/world-news/italy...
239,37,2025-08-06-11-49-45 +0000,nypost,Virgin Atlantic passenger threatens to rape fl...,https://nypost.com/2025/08/06/world-news/staff...
73,36,2025-08-06-22-00-21 +0000,nypost,Possible human skeleton found next to Jackie R...,https://nypost.com/2025/08/06/us-news/possible...
115,32,2025-08-06-20-03-00 +0000,wsj,President Trump’s tariff rollout has taken num...,https://www.wsj.com/economy/trade/trump-tariff...
13,30,2025-08-07-00-49-04 +0000,nypost,Anti-Israel activist Mahmoud Khalil argues Oct...,https://nypost.com/2025/08/06/us-news/mahmoud-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
